# # AWS CDK Deployment – Part 4 - Python Version (no cloudshell)
This step defines the infrastructure for the data pipeline using AWS CDK (Python) to stand up and wire everything for automated ingestion and processing of the BLS + population datasets.

**[View CloudShell/CDK Logs — Part 4 (Sanitized)](https://github.com/ScottySchmidt/AWS_DataEngineer_API/tree/main/docs/part4)**  
↑ That link shows the CloudShell based deploy for reference; this section documents the pure Python/CDK approach.

Uses AWS CDK (Python) to deploy the pipeline infrastructure:
- Lambda, S3, and SQS resources
- A daily ingest job (Parts 1 & 2)
- Automatic processing (Part 3) when new files land in S3

### Pipeline Flow
1. Deploy infrastructure with CDK (S3, Lambdas, SQS, EventBridge, IAM).
2. EventBridge triggers the Ingest Lambda daily.
3. Ingest writes fresh datasets to S3.
4. S3 emits object-created events to SQS (filtered for your files).
5. SQS invokes the Report Lambda to process/aggregate and write outputs.
6. CloudWatch Logs capture logs/metrics for both Lambdas.

In [1]:
import boto3
import os
import requests
import hashlib
import json
from kaggle_secrets import UserSecretsClient

# Load AWS secrets
secrets = UserSecretsClient()
API_KEY = secrets.get_secret("BLS_API_KEY")
AWS_ACCESS_KEY_ID = secrets.get_secret("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = secrets.get_secret("AWS_SECRET_ACCESS_KEY")
AWS_REGION = secrets.get_secret("AWS_REGION")
BUCKET_NAME = secrets.get_secret("BUCKET_NAME")

# Setup AWS session and S3
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION
)
s3 = session.client("s3")

# Test connection WITHOUT revealing keys
try:
    response = s3.list_objects_v2(Bucket=BUCKET_NAME)
    num_files = response.get('KeyCount', 0)
    print("S3 connection successful. Bucket contains: ", num_files)
except Exception as e:
    print("S3 connection failed: ", e)

S3 connection successful. Bucket contains:  6


In [2]:
import os
from kaggle_secrets import UserSecretsClient

secrets = UserSecretsClient()

# Pull from Kaggle Secrets and mirror into environment vars
for key in ["AWS_ACCESS_KEY_ID", "AWS_SECRET_ACCESS_KEY", "AWS_REGION", "BUCKET_NAME"]:
    v = secrets.get_secret(key)
    if v:
        os.environ[key] = v

# If only AWS_REGION is set, mirror it to AWS_DEFAULT_REGION for libraries that expect that
if os.environ.get("AWS_REGION") and not os.environ.get("AWS_DEFAULT_REGION"):
    os.environ["AWS_DEFAULT_REGION"] = os.environ["AWS_REGION"]

print("Exported Kaggle secrets to env vars.")

Exported Kaggle secrets to env vars.


In [3]:


REGION = os.environ.get("AWS_DEFAULT_REGION", "us-east-1")
sqs = boto3.client("sqs", region_name=REGION)

resp = sqs.create_queue(
    QueueName="JsonWriteQueue",
    Attributes={"VisibilityTimeout": "300"}  # seconds
)
QUEUE_URL = resp["QueueUrl"]
QUEUE_ATTR = sqs.get_queue_attributes(QueueUrl=QUEUE_URL, AttributeNames=["QueueArn"])
QUEUE_ARN = QUEUE_ATTR["Attributes"]["QueueArn"]

print("SQS ready")
print("ARN ready")

SQS ready
ARN ready


In [4]:
#LAMBDA SMOKE TEST: 
lam = session.client("lambda")
fns = lam.list_functions()['Functions']
names = []
for function in fns:
    name = function["FunctionName"]
    names.append(name)
print("Functions:", names)

Functions: ['BlsPipelineStack-BlsReportFn1ABE5E21-0qZIzspWVVtL', 'CFN-SM-IM-Lambda-catalog-DelayLambda-GH20qvggV0UO', 'BlsPipelineStack-BlsIngestFnCD0BBA37-KqHbWeaT1WT3', 'bls-ingest', 'CFGetCatalogRoles', 'RearcBLSLambda', 'CFGetDefaultVpcIdTut', 'MyFirstLambda', 'S3FileProcessorLambda', 'GitHubWebhookLambda', 'BlsPipelineStack-BucketNotificationsHandler050a058-iFZUwiLm9Toh', 'CFEnableSagemakerProjectsTut']


### Conclusion
That wraps up Part 4 – Infrastructure as Code & the automated pipeline.

Here’s what we confirmed:
The stack deployed successfully with no errors.
The ingestion Lambda (combining Parts 1 & 2) ran and pulled the data.
New uploads to S3 triggered the SQS queue as expected.
The reporting Lambda processed those messages and logged the results.
Final datasets are safely stored in S3 (see screenshot above).

[View GitHub Repository with All Parts](https://github.com/ScottySchmidt/AWS_DataEngineer_API)